# Mockup Version 1

- Mock-up the workflow from MIS global summit in an environment like EvidenceCare's Epic App. Due January 1.
    - ☑️ List content for our app (Due by Nov 23) TJM
    - ☑️ Define the workflow between user, website, and service (Due by Nov 23) TJM & NS workshop session
    - ☑️ Define interface between UI and service (Due by Nov 30) TJM & NS workshop session
    - ☑️ Include procedures predictor (Due by Dec 14) TJM
    - ☑️ Create a static page layout (Due by Dec 7) NS
    - ☑️ Flask configuration (due by Dec 1) TJM 
    - Have shortest path algorithms produce sentences of node-relationship-node, where you can click on a word and it highlights the node in the graph visualization
    - Tim give Nikesh an explanation of the score value which he can add to the prototype
    - Tim incorporate Nikesh's fetchData and flask code into current code
    - Write & test the code for webpage (Due by Dec 23) TJM & NS
    - Provide instructions, documentation, supporting information to make it (Due by Dec 23) BED
    - 🟢 Create a page on mi1.ai that introduces the POC
    - 🟢 Create a slide that describes what is going on under the hood in the poc
    - 🟢 Create an idealized version of visualization, which can be inserted into a screen-recorded demo right after showing the live graph view
    

In [1]:
import pandas as pd
import time

In [12]:
# import getpass
# password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j','NikeshIsCool'))
session=driver.session()

In [4]:
# Get a list of autocomplete terms
startingtext = 'Res'
query = '''
MATCH (c1:Concept)
WHERE c1.term STARTS WITH '{startingtext}' AND c1.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"] 
RETURN DISTINCT(c1.cui) AS `Known_CUI`, c1.term AS `Known_Problem`
'''.format(startingtext=startingtext)
data = session.run(query)
autocomplete_problems_prompts = pd.DataFrame([dict(record) for record in data])
autocomplete_problems_prompts

,Known_CUI,Known_Problem
0,C4046002,Research Related Injuries
1,C4046002,Research-Related Injuries
2,C4046002,Research-Related Injury
3,C0862970,Resectable Bile Duct Adenocarcinoma
4,C0854792,Resectable Bile Duct Cancer
...,...,...
985,C3281186,Restrictive ventilation disorder due to severe...
986,C0036351,Restzustand
987,C0036351,Restzustand (schizophrenic)
988,C0036351,"Restzustand, schizophrenic"


In [94]:
    cui_prob_list = ['C1565489', 'C0085762']
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r, c
    RETURN c.cui AS `Known_CUI`, c.term AS `Known_Problem`, ord.cui AS `CUI`, ord.term AS `Problem`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities_NLP = pd.DataFrame([dict(record) for record in data])
    likely_comorbidities_NLP

,Known_CUI,Known_Problem,CUI,Problem,Score
0,C0085762,Alcohol abuse,C0220982,Ketoacidosis,0.61598
1,C0085762,Alcohol abuse,C0149521,"Pancreatitis, Chronic",0.56788
2,C0085762,Alcohol abuse,C0151723,Hypomagnesemia,0.51528
3,C0085762,Alcohol abuse,C0236663,Alcohol withdrawal syndrome,0.51037
4,C0085762,Alcohol abuse,C1963823,Hyperlipasaemia,0.46902
5,C0085762,Alcohol abuse,C0438696,Suicidal,0.46201
6,C0085762,Alcohol abuse,C0747752,Polysubstance abuse,0.45843
7,C0085762,Alcohol abuse,C0162316,Iron deficiency anemia,0.44179
8,C0085762,Alcohol abuse,C3159311,BORNHOLM EYE DISEASE,0.42945
9,C0085762,Alcohol abuse,C0232498,Abdominal tenderness,0.42369


In [18]:
# get a CSV of all problems in the database that have enough data to potentially return robust results
query = '''
MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r, c
WHERE Score > 20
RETURN distinct(c.term) AS Problem, c.cui'''
data = session.run(query)

# Convert the neo4j object into a dataframe
common_problems_df = pd.DataFrame([dict(record) for record in data])

In [21]:
common_problems_df.sort_values(by='Problem', inplace=True)
common_problems_df.Problem.to_list()

['Abnormally opaque structure (morphologic abnormality)',
 'Acidosis',
 'Acidosis, Lactic',
 'Acidosis, Respiratory',
 'Actinic cheilitis',
 'Acute respiratory failure',
 'Aggressive Periodontitis',
 'Agitation',
 'Alcohol Withdrawal Delirium',
 'Alcohol abuse',
 'Alcohol withdrawal syndrome',
 'Alkalosis, Respiratory',
 'Aneurysm',
 'Apnea',
 'Ascites',
 'Back Pain',
 'Blood Clot',
 'Bronchospasm',
 'Cardiac Arrhythmia',
 'Cerebrovascular accident',
 'Chest Pain',
 'Cholangiocarcinoma',
 'Chronic Kidney Diseases',
 'Chronic back pain',
 'Confusion',
 'Contraction (finding)',
 'Coughing',
 'Dentatorubral-Pallidoluysian Atrophy',
 'Desmoplastic infantile ganglioglioma',
 'Diabetes Mellitus, Non-Insulin-Dependent',
 'Diabetic Ketoacidosis',
 'Diastolic dysfunction',
 'Disease',
 'Disease Progression',
 'Dry skin',
 'Dyspnea',
 'Encephalitis',
 'Epilepsy',
 'Esophageal Varices',
 'Fatigue',
 'Febrile Neutropenia',
 'Flare',
 'Fluid overload',
 'Gastrointestinal Hemorrhage',
 'Gastroparesi

In [26]:
# Trim out problems that seem to be spurious
spurious = ['Abnormally opaque structure (morphologic abnormality)', 'Actinic cheilitis', 'Aggressive Periodontitis', 'Dentatorubral-Pallidoluysian Atrophy', 'Desmoplastic infantile ganglioglioma',  'Disease', 'Disease Progression', 'Flare', 'Obstruction', 'POPLITEAL PTERYGIUM SYNDROME, LETHAL TYPE',  'Pathological Dilatation', 'Pathology processes', 'Sore to touch', 'Xanthomatosis, Cerebrotendinous',  'diastolic congestive heart failure', 'gastrointestinal gas', 'physiologic resolution']
common_problems_df = common_problems_df[~common_problems_df.Problem.isin(spurious)]

In [27]:
# Write out to CSV
common_problems_df.to_csv('common_problems.csv', index=False, encoding='utf-8')

## Modify existing functions for the new data model
### Input a list of CUIs for known problems, output possible additional problems with their CUIs and odds ratios in a dataframe
Note: This query relies on the OCCURS_WITH relationship, which was created in the [Virtual_Population](Documents/P2_Prototype/Virtual_Population.ipynb) notebook under the section "Using Conditional Probability instead of Odds Ratio."

In [12]:
def PotentialComorbidities(cui_prob_list):
    
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"]
    WITH round(r.co_occurrance_probability, 5) AS Score, ord, r, c
    RETURN c.cui AS `Known_CUI`, c.term AS `Known_Problem`, ord.cui AS `Likely_CUI`, ord.term AS `Likely_Problem`, Score
    ORDER BY r.co_occurrance_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities = pd.DataFrame([dict(record) for record in data])
    
    return likely_comorbidities

In [13]:
# Test the function
start_time = time.time()

cui_prob_list = ['C1565489', 'C0085762']
PotentialComorbidities = PotentialComorbidities(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")
PotentialComorbidities

Total runtime: 0.0643320083618164 seconds


,Known_CUI,Known_Problem,Likely_CUI,Likely_Problem,Score
0,C0085762,Alcohol abuse,C0162316,Iron deficiency anemia,None
1,C0085762,Alcohol abuse,C0948268,Hemodynamic instability,None
2,C0085762,Alcohol abuse,C0394996,Acute alcoholic intoxication,None
3,C0085762,Alcohol abuse,C4761312,Nutritional Deficiency,None
4,C0085762,Alcohol abuse,C0220982,Ketoacidosis,None
5,C0085762,Alcohol abuse,C0677042,Pathology processes,None
6,C0085762,Alcohol abuse,C0149521,"Pancreatitis, Chronic",None
7,C0085762,Alcohol abuse,C0151723,Hypomagnesemia,None
8,C0085762,Alcohol abuse,C0338908,Mixed anxiety and depressive disorder,None
9,C1565489,Renal Insufficiency,C2826321,Refractory Thrombocytopenia,None


### Input 1 problem, output the most likely orders
Note: This query relies on the OCCURS_WITH relationship, which was created in the [Virtual_Population](Documents/P2_Prototype/Virtual_Population.ipynb) notebook under the section "Using Conditional Probability instead of Odds Ratio."

In [9]:
def LikelyOrders(cui_prob_list):
    
    # Find prescriptions associated with the input problem    
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Clinical Drug']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r
    RETURN ord.cui AS `Code`, ord.term AS `Order`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_rx = pd.DataFrame([dict(record) for record in data]).head(10)
    
    # Find abnormal labs associated with the input problem
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Clinical Attribute']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r
    RETURN ord.cui AS `Code`, ord.description AS `Order`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_lab = pd.DataFrame([dict(record) for record in data]).head(10)
    
    # Find procedures associated with the input problem
    query = '''
    MATCH p=(ord:D_Icd_Procedures)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list}
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r
    RETURN ord.icd9_code AS `Code`, ord.long_title AS `Order`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_procedure = pd.DataFrame([dict(record) for record in data]).head(10)
    
    return orders_likely_rx, orders_likely_lab, orders_likely_procedure

In [ ]:
MATCH (source:Concept {cui:'C0022661'})
MATCH (destination:Concept {cui:'C0975120'})
CALL gds.alpha.shortestPath.stream({
    startNode: source,
    endNode: destination,
    nodeProjection: 
})

In [26]:
start_time = time.time()

cui_prob_list = ['C0022661']
orders_likely_rx, orders_likely_lab, orders_likely_procedure = LikelyOrders(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 0.023564577102661133 seconds


In [27]:
orders_likely_rx

,Code,Order,Score
0,C0975120,calcitriol 0.00025 MG Oral Capsule,0.62000
1,C1967412,sevelamer carbonate 800 MG Oral Tablet [Renvela],0.60450
2,C1951501,sodium polystyrene sulfonate 250 MG/ML Oral Su...,0.58420
3,C0706911,1 ML epoetin alfa 4000 UNT/ML Injection [Procrit],0.44712
4,C4082775,150 ML Glucose 50 MG/ML Injection,0.42971
5,C0710995,purified protein derivative of tuberculin 50 U...,0.39966
6,C0691839,sodium bicarbonate 650 MG Oral Tablet,0.37536
7,C0978650,midodrine hydrochloride 5 MG Oral Tablet,0.35985
8,C1169852,piperacillin 2000 MG / tazobactam 250 MG Injec...,0.35321
9,C0975350,cefepime 1000 MG Injection,0.33979


In [28]:
orders_likely_lab

,Code,Order,Score
0,C1544491,Macrophage in Ascites,0.77165
1,C0549816,Lymphocytes in Other Body Fluid,0.76913
2,C0942476,Polys in Other Body Fluid,0.71643
3,C0942444,Lymphocytes in Ascites,0.70352
4,C0942425,"RBC, Ascites in Ascites",0.68548
5,C0945358,"WBC, Ascites in Ascites",0.67514
6,C0942450,Monocytes in Ascites,0.66127
7,C0942478,Polys in Ascites,0.64904
8,C0365034,Protein/Creatinine Ratio in Urine,0.63874
9,C0368042,pH in Urine,0.59096


In [29]:
orders_likely_procedure

,Code,Order,Score
0,3995,Hemodialysis,0.75877
1,3895,Venous catheterization for renal dialysis,0.70488
2,4513,Other endoscopy of small intestine,0.26072
3,3324,Closed [endoscopic] biopsy of bronchus,0.24204
4,3893,"Venous catheterization, not elsewhere classified",0.22290
5,9904,Transfusion of packed cells,0.22250
6,9604,Insertion of endotracheal tube,0.20985


In [25]:
orders_likely_rx['Type'] = 'Prescription'
orders_likely_lab['Type'] = 'Lab'
orders_likely_procedure['Type'] = 'Procedure'
AssocOrders = pd.concat([orders_likely_rx, orders_likely_lab, orders_likely_procedure])
AssocOrders.sort_values('Score', ascending = False, inplace = True)
AssocOrders = AssocOrders.head(10)
AssocOrders

,Code,Order,Score,Type
0,C1544491,Macrophage in Ascites,0.77165,Lab
1,C0549816,Lymphocytes in Other Body Fluid,0.76913,Lab
0,3995,Hemodialysis,0.75877,Procedure
2,C0942476,Polys in Other Body Fluid,0.71643,Lab
1,3895,Venous catheterization for renal dialysis,0.70488,Procedure
3,C0942444,Lymphocytes in Ascites,0.70352,Lab
4,C0942425,"RBC, Ascites in Ascites",0.68548,Lab
5,C0945358,"WBC, Ascites in Ascites",0.67514,Lab
6,C0942450,Monocytes in Ascites,0.66127,Lab
7,C0942478,Polys in Ascites,0.64904,Lab
